# Ex 1
The function below is able to scrape all the reviews contained on a Trustpilot page (whose link is passed as parameter) and the store these information in a dictionary. Modify the function in order to scrape the <em>name</em> of the reviewer, the <em>location</em>, the <em>number of reviews</em> and the <em>title</em> of them. Moreover, scrape the replies if they exist, otherwise set the value at None. Save all the information in a <em>.csv</em> file (the path of the file must be the second argument of the function). Divide all the information with a semicolon ";".

In [ ]:
def trust_juicer(u):
    source_ = requests.get(u).text
    res_ = BeautifulSoup(source_).find_all("article")
    d = {}
    for i,t in enumerate(res_):
        try:
            tempd = {}
            text = t.find("p").get_text()
            label = t.find("div", class_= "star-rating star-rating--medium").find("img").attrs["alt"][0]

            tempd["text"] = text.strip()
            tempd["label"] = label

            d[i] = tempd
        except Exception as e:
            print(i, e)
    
    return d

In [ ]:
import requests
from bs4 import BeautifulSoup 

tp_url = "https://www.trustpilot.com/review/www.nzymes.com"
src = requests.get(tp_url).text
soup = BeautifulSoup(src)

def trust_juicer(u, path):
    source_ = requests.get(u).text
    soup = BeautifulSoup(source_)
    res_ = soup.find_all("article")
    file = open(path, "w", encoding="utf-8")
    d = {}
    for i,t in enumerate(res_):
        try:
            tempd = {}
            text = t.find("p").get_text()
            label = t.find("div", class_= "star-rating star-rating--medium").find("img").attrs["alt"][0]
            name = t.find("div", class_="consumer-information__name").get_text()
            location = t.find("div", class_="consumer-information__location").find("span").get_text()
            n_of_reviews = t.find("div", class_="consumer-information__review-count").find("span").get_text()
            title = t.find("h2", class_="review-content__title").get_text()
            
            try:
                reply = t.find("div", class_="brand-company-reply__content").get_text()
            except Exception as e:
                reply = "None"
            
            tempd["location"] = location
            tempd["n_of_reviews"] = n_of_reviews
            tempd["name"] = name.strip()
            tempd["title"] = title.strip()
            tempd["text"] = text.strip()
            tempd["label"] = label
            tempd["reply"] = reply.strip()
            
            d[i] = tempd
            
            line = tempd["name"] + ";" + tempd["location"] + ";" + tempd["n_of_reviews"] + ";" + tempd["title"] + ";" + tempd["text"] + ";" + tempd["label"] + ";" +  tempd["reply"]
            file.write(line + "\n")
            
        except Exception as e:
            print(i, e)
        
    return d

In [ ]:
trust_juicer(tp_url, "trust_data.csv")

# Ex 2

Lets make things more complicated. Start with the code from exercise 1 and make sure you get the link to the next page. Then, modify and use the function from exercise 1 to extract five pages of reviews. Find a way to save all the information in a single dictionary and in a unique .csv file

In [ ]:
import requests
from bs4 import BeautifulSoup 

tp_url = "https://www.trustpilot.com/review/www.nzymes.com"
src = requests.get(tp_url).text
soup = BeautifulSoup(src)

def trust_juicer(u, path, index):
    source_ = requests.get(u).text
    soup = BeautifulSoup(source_)
    res_ = soup.find_all("article")
    file = open(path, "w", encoding="utf-8")
    next_page = soup.find("a", class_="next-page")['href']
    d = {}
    for t in res_:
        try:
            tempd = {}
            text = t.find("p").get_text()
            label = t.find("div", class_= "star-rating star-rating--medium").find("img").attrs["alt"][0]
            name = t.find("div", class_="consumer-information__name").get_text()
            location = t.find("div", class_="consumer-information__location").find("span").get_text()
            n_of_reviews = t.find("div", class_="consumer-information__review-count").find("span").get_text()
            title = t.find("h2", class_="review-content__title").get_text()
            
            try:
                reply = t.find("div", class_="brand-company-reply__content").get_text()
            except Exception as e:
                reply = "None"
            
            tempd["location"] = location
            tempd["n_of_reviews"] = n_of_reviews
            tempd["name"] = name.strip()
            tempd["title"] = title.strip()
            tempd["text"] = text.strip()
            tempd["label"] = label
            tempd["reply"] = reply.strip()
            
            d[index] = tempd
            index += 1
            
            line = tempd["name"] + ";" + tempd["location"] + ";" + tempd["n_of_reviews"] + ";" + tempd["title"] + ";" + tempd["text"] + ";" + tempd["label"] + ";" +  tempd["reply"]
            file.write(line + "\n")
            
        except Exception as e:
            print(i, e)
        
    return d, next_page, index

In [ ]:
trust_data = {}
index = 0
d, next_page, index =  trust_juicer(tp_url, "trust_data.csv", index)
trust_data.update(d)
for i in range(4):
    d, next_page, index =  trust_juicer("https://www.trustpilot.com" + next_page, "trust_data.csv", index)
    trust_data.update(d)
    
trust_data

# Ex 3

Rev's website contains transcripts of oral speeches of various kinds. Create a function that takes a url as an argument and that is able to extract a complete dialogue. Once the text is obtained, use regular expressions to clean it up. In particular, remove:
- the name of the interlocutor;
- time indications;
- numbers (watch out for percentages and currencies);
- alphanumeric codes. 

In addition, discard all sentences with less than 100 characters. Save the cleaned text into a text file. 

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

rev_url = "https://www.rev.com/blog/transcripts/apple-event-september-2021-transcript-new-iphone-13-apple-watch-ipads"

def rev_scraper(url_):
    source = requests.get(url_)
    soup = BeautifulSoup(source.text)

    # scraping all the dialogue
    text = soup.find("div", class_="fl-callout-text").find_all("p")
    text = [p.get_text() for p in text]
    
    file = open("cleaned_text.txt", "w", encoding="utf-8")
    for p in text:
        if len(p) > 100:
            speech = p
            # removing name of the interlocutor and time indications 
            if re.search(r"^\w+(\s\w+)?:\s\(.*\)\n", speech, re.IGNORECASE):
                speech = re.sub(r"^\w+(\s\w+)?:\s\(.*\)\n", "", speech, re.IGNORECASE)
            # removing numbers 
            if re.search(r"\$?\d+%?(\.\d+)?(\s|,)", speech):
                speech = re.sub(r"\$?\d+%?(\.\d+)?(\s|,|\.)", "", speech)
            # removing alphanumeric codes (ex. "5G", "4K")
            if re.search(r"([A-z]+[\d]+)+([A-z]+)?|([\d]+[A-z]+)+([\d]+)?", speech):
                speech = re.sub(r"([A-z]+[\d]+)+([A-z]+)?|([\d]+[A-z]+)+([\d]+)?", "", speech)    
            file.write(speech + "\n\n")
            
rev_scraper(rev_url)

# Ex 4
Let us make a slight modification to the function of the previous exercise: after extracting all the speeches from the page's HTML, the function will save them in a dictionary. This dictionary will have the name of the interlocutor as a key. The dictionary values must be a tuple whose first element is the moment of the recording when the interlocutor is speaking and the second element is the transcription of what is being said. 

In [ ]:
import re
import requests
from bs4 import BeautifulSoup

rev_url = "https://www.rev.com/blog/transcripts/apple-event-september-2021-transcript-new-iphone-13-apple-watch-ipads"

def rev_scraper(url_):
    source = requests.get(url_)
    soup = BeautifulSoup(source.text)

    # scraping all the dialogue
    text = soup.find("div", class_="fl-callout-text").find_all("p")
    text = [p.get_text() for p in text]
    
#     file = open("cleaned_text.txt", "w", encoding="utf-8")
    data = {}
    for p in text:
        if len(p) > 100:
            # splitting the speech using the time indication
            # type(speech) : list 
            speech = re.split(r"(:\s\(.*\)\n)", p)
            speacker_name = speech[0]
            time_indication = speech[1] 
            transcript = speech[2]
            time_indication = re.sub(r":\s\((.*)\)\n", r"\1", time_indication)
    
            # removing numbers 
            if re.search(r"\$?\d+%?(\.\d+)?(\s|,)", transcript):
                transcript = re.sub(r"\$?\d+%?(\.\d+)?(\s|,|\.)", "", transcript)
            # removing alphanumeric codes (ex. "5G", "4K")
            if re.search(r"([A-z]+[\d]+)+([A-z]+)?|([\d]+[A-z]+)+([\d]+)?", transcript):
                transcript = re.sub(r"([A-z]+[\d]+)+([A-z]+)?|([\d]+[A-z]+)+([\d]+)?", "", transcript)
            
            if speacker_name in data:
                data[speacker_name].append((time_indication,transcript))
            else:
                data[speacker_name] = [(time_indication,transcript)]
    return data
rev_scraper(rev_url)

# Ex 5

Build your calculator using regular expressions. Thus, write a function that takes as input an operation defined as a string of the kind: "10+12", "10*12" etc. Using regular expressions, extract the operands and figure out which operation should be applied based on the symbol that divides them. The function must return the result. 
Using the function you have just defined, use the strings contained in the operations list in order to obtain a list identical to the results one.

In [ ]:
def addition(a,b):
    return a+b


def subtraction(a,b):
    return a-b


def division(a,b):
    try: 
        return a/b
    except ZeroDivisionError as e:
        print(f"You cannot divide a number by 0!")
        
        
def multiplication(a,b):
    return a*b

def calculator(operation):
    numbers = re.split(r"\D", operation)
    if re.search("\+", operation):
        return addition(int(numbers[0]),int(numbers[1]))
    if re.search("\-", operation):
        return subtraction(int(numbers[0]),int(numbers[1]))
    if re.search(":", operation):
        return division(int(numbers[0]),int(numbers[1]))
    if re.search("\*", operation):
        return multiplication(int(numbers[0]),int(numbers[1]))

In [ ]:
operations = ["10+12","12-10","3*2","6:3"]
results = [calculator(op) for op in operations]